# Use a large language model to cluster sticky notes

### Scenario
Imagine you are a team lead and you are taking your team through a reflection activity.  Your team has used a mural to record their feedback.  Now, you need to cluster the sticky notes in the mural for further analysis.

Notebook sections:

**Section A - Explore prompting large languge models**
- [Step 1: Set up IBM watsonx.ai foundation model Python library prerequisites](#step1)
- [Step 2: Create a function for prompting a model to identify top themes in messages](#step2)
- [Step 3: Create a function for prompting a model to classify a message](#step3)

**Section B - Set up your mural**
- [Step 4: Set up MURAL prerequisites](#step4)
- [Step 5: Read feedback messages from your mural](#step5)

**Section C - Cluster sticky notes in the mural**
- [Step 6: Identify top themes in sticky notes](#step6)
- [Step 7: Classify sticky notes](#step7)
- [Step 8: Move sticky notes into labelled boxes in the mural](#step8)

When you run this notebook, your mural will look something like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-17_llm-cluster_01.png" width="50%" title="Image of a mural" />

# Section A - Explore prompting large languge models

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-17_llm-cluster_02.png" width="60%" title="Image of a mural" />

<a id="step1"></a>
## Step 1: Set up IBM watsonx.ai foundation model Python library prerequisites
Before you can prompt a foundation model in watsonx.ai, you must perform the following setup tasks:
- 1.1 Create an instance of the Watson Machine Learning service
- 1.2 Associate the Watson Machine Learning instance with the current project
- 1.3 Create an IBM Cloud API key
- 1.4 Create a credentials dictionary for Watson Machine learning
- 1.5 Look up the current project ID

### 1.1 Create an instance of the Watson Machine Learning service
If you don't already have an instance of the IBM Watson Machine Learning service, you can create an instance of the service from the IBM Cloud catalog: <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank">Watson Machine Learning service</a>

### 1.2 Associate an instance of the Watson Machine Learning service with the current project
The _current project_ is the project in which you are running this notebook.

If an instance of Watson Machine Learning is not already associated with the current project, follow the instructions in this topic to do so: <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=wx&audience=wdp" target="_blank">Adding associated services to a project</a>

### 1.3 Create an IBM Cloud API key
Create an IBM Cloud API key by following these instruction: <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key" target="_blank">Creating an IBM Cloud API key</a>

Then paste your new IBM Cloud API key in the code cell below.

In [1]:
g_cloud_apikey = ""

### 1.4 Create a credentials dictionary for Watson Machine learning
See: [Authentication](https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication)

In [2]:
# See: https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication
g_wml_credentials = { 
    "url"    : "https://us-south.ml.cloud.ibm.com", 
    "apikey" : g_cloud_apikey
}

### 1.5 Look up the current project ID
The _current project_ is the project in which you are running this notebook.  You can get the ID of the current project programmatically by running the following cell.

In [3]:
import os

g_project_id = os.environ["PROJECT_ID"]

<a id="step2"></a>
## Step 2: Create a function for prompting a model to identify top themes in sticky notes
- 2.1 Experiment in Prompt Lab
- 2.2 Specify your selected model ID, prompt parameters, and prompt text template
- 2.2 Define a function to identify themes

### 2.1 Experiment in Prompt Lab 
The Prompt Lab in watsonx.ai is a graphical interface for experimenting with prompting foundation models.

Experiment in Prompt Lab to discover what works best:
- Which model returns ideal results
- What parameter settings (eg. decoding) work best
- What prompt text causes the model to respond the way you want

See: [Prompt Lab](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-prompt-lab.html?context=wx&audience=wdp)

### 2.2 Specify your selected model_id, prompt parameters, and prompt text template
In the following three cells, there are example model ID, prompt parameters, and a prompt text template you can use.

Replace any of these with values you discovered while experimenting in Prompt Lab.

In [4]:
# Example models:
#
# google/flan-ul2
# google/flan-t5-xxl
# bigscience/mt0-xxl
# eleutherai/gpt-neox-20b
# ibm/granite-13b-instruct-v1

g_themes_model_id = "meta-llama/llama-2-70b-chat"

In [5]:
g_themes_prompt_parameters = {
    "decoding_method" : "greedy",
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 20,
    "stop_sequences"  : [ "\n\n" ]
}

**NOTE**

In the following example template, notice the use of `%s` as a placeholder for the messages in the sticky notes. 

If you replace this template with a prompt you discovered through your experiments in Prompt Lab, remember to include a placeholder for the messages.

In [6]:
g_themes_prompt_template = """The following meals fall into three broad themes:
- Spaghetti with meatballs
- Vegetarian sub
- Shrimp pad thai
- Fish fingers
- Falafel
- Steak and kidney pie

Three themes in the list of meals:
- Meat
- Vegetarian
- Seafood


The following animals fall into three broad themes:
- Cow
- Chicken
- Dog
- Giraffe
- Gerbil
- Elephant

Three themes in the list of animals:
- Pet
- Farm
- Wild


The following process feedback ideas fall into three broad themes:
%s

Think past what each idea mentions and consider larger patterns across the ideas.

Three themes that get at the essence of the list of process feedback ideas:
"""

def createThemesPromptText( messages_arr ):
    messages_str = "- " + "\n- ".join( messages_arr )
    return g_themes_prompt_template % ( messages_str )

In [ ]:
g_test_messages_arr = [ 
    "Our daily stand-ups take too long.  Let's stop doing those.",
    "Can we please stop having to sing the national anthem at the start of every day?!", 
    "Stop having all-hands meetings, because they don't add any value.", 
    "Stop forcing everyone to attend the all-hands in person", 
    "I like the \"updates\" Slack channel - very efficient",
    "Loving the on-site day care",
    "Please let's continue the THINK-Thursday meeting-free afternoon!",
    "The monthly performance awards are a nice recognition",
    "I think we should record our meetings so they could be played back",
    "At my friend's company, they have a lottery for plum parking spots.  We should do something similar!",
    "We should take meeting minutes",
    "Because of these power outages, we should invest in uninterrupted power supply infrastructure"
]
g_themes_test_prompt_text = createThemesPromptText( g_test_messages_arr )
print( g_themes_test_prompt_text )

### 2.3 Define a function to identify themes
You can prompt foundation models in IBM watsonx.ai programmatically using the foundation models Python library.

See:
- <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-python-lib.html?context=wx&audience=wdp" target="_blank">Introduction to the foundation models Python library</a>
- <a href="https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html" target="_blank">Foundation models Python library reference</a>

Test the library:

In [8]:
from ibm_watson_machine_learning.foundation_models import Model
import json

themes_test_model = Model( g_themes_model_id, g_wml_credentials, g_themes_prompt_parameters, g_project_id )

themes_test_response = themes_test_model.generate( g_themes_test_prompt_text )

print( "themes_test_response:\n" + json.dumps( themes_test_response, indent=3 ) )

themes_test_response:
{
   "model_id": "meta-llama/llama-2-70b-chat",
   "created_at": "2023-11-15T00:00:58.117Z",
   "results": [
      {
         "generated_text": "- Efficiency\n- Culture\n- Communication\n\n",
         "generated_token_count": 13,
         "input_token_count": 389,
         "stop_reason": "stop_sequence"
      }
   ],
   "system": {
      "warnings": [
         {
            "message": "This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL. URL: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx",
            "id": "disclaimer_warning"
         }
      ]
   }
}


In [9]:
import re

def generate( model_id, prompt_parameters, prompt_text, b_debug=False ):
    model = Model( model_id, g_wml_credentials, prompt_parameters, g_project_id )
    raw_response = model.generate( prompt_text )
    if b_debug:
        print( "\nraw_response:\n" + json.dumps( raw_response, indent=3 ) )
    if ( "results" in raw_response ) \
       and ( len( raw_response["results"] ) > 0 ) \
       and ( "generated_text" in raw_response["results"][0] ):
        return raw_response, raw_response["results"][0]["generated_text"]
    else:
        print( "\nThe model failed to generate an answer" )
        print( "\nDebug info:\n" + json.dumps( raw_response, indent=3 ) )
        return raw_response, ""

def identifyThemes( model_id, prompt_parameters, messages_arr, b_debug=False ):
    prompt_text = createThemesPromptText( messages_arr )
    if b_debug:
        print( "Prompt:\n--------------------------START")
        print( prompt_text + "--------------------------END")
    raw_response, generated_output = generate( model_id, prompt_parameters, prompt_text, b_debug )
    class_names_str = re.sub( r"^\s*\-\s*", "", generated_output )
    class_names_str = re.sub( r"\s+$", "", class_names_str )
    class_names_arr = re.split( r"\s*\n+\-\s*", class_names_str )
    class_names_arr = [ item.strip().capitalize() for item in class_names_arr ]
    return class_names_arr

In [ ]:
g_test_themes_arr = identifyThemes( g_themes_model_id, g_themes_prompt_parameters, g_test_messages_arr, b_debug=True )

print( "\ng_test_themes_arr:\n" + json.dumps( g_test_themes_arr, indent=3 ) )

<a id="step3"></a>
## Step 3: Create a function for prompting a model to classify sticky notes
- 3.1 Experiment in Prompt Lab again
- 3.2 Specify your selected model ID, prompt parameters, and prompt text template
- 3.2 Define a function to perform classification

### 3.1 Experiment in Prompt Lab again
Experiment in Prompt Lab to discover what works best:
- Which model returns ideal results
- What parameter settings (eg. decoding) work best
- What prompt text causes the model to respond the way you want

### 3.2 Specify your selected model_id, prompt parameters, and prompt text template
In the following three cells, there are example model ID, prompt parameters, and a prompt text template you can use.

Replace any of these with values you discovered while experimenting in Prompt Lab.

In [11]:
# Example models:
#
# google/flan-ul2
# google/flan-t5-xxl
# bigscience/mt0-xxl
# eleutherai/gpt-neox-20b
# ibm/granite-13b-instruct-v1

g_classify_model_id = "google/flan-t5-xxl"

In [12]:
g_classify_prompt_parameters = {
    "decoding_method" : "greedy",
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 20
}

**NOTE**

In the following example template, notice the use of `%s` as a placeholder for the class names and message text. 

If you replace this template with a prompt you discovered through your experiments in Prompt Lab, remember to include a placeholder for the class names and one for the message text.

In [13]:
g_classify_prompt_template = """Classify the message into one of three classes: %s

If the message doesn't fit any of the classes, say "Other"

Message: %s
Class: 
"""

def createClassifyPromptText( class_names_arr, message_text ):
    class_name_str = ", ".join( class_names_arr )
    return g_classify_prompt_template % ( class_name_str, message_text )

In [14]:
g_classify_test_message = "I think we should record our meetings so they could be played back"
g_classify_test_prompt_text = createClassifyPromptText( g_test_themes_arr, g_classify_test_message )
print( g_classify_test_prompt_text )

Classify the message into one of three classes: Efficiency, Culture, Communication

If the message doesn't fit any of the classes, say "Other"

Message: I think we should record our meetings so they could be played back
Class: 



### 3.3 Define a function to perform classification

In [15]:
def classifyMessage( model_id, prompt_parameters, class_names_arr, message_text, b_debug=False ):
    prompt_text = createClassifyPromptText( class_names_arr, message_text )
    if b_debug:
        print( "Prompt:\n--------------------------START")
        print( prompt_text + "--------------------------END")
    raw_response, generated_output = generate( model_id, prompt_parameters, prompt_text, b_debug )
    class_name = generated_output.strip().capitalize()
    return class_name

In [ ]:
classifyMessage( g_classify_model_id, g_classify_prompt_parameters, g_test_themes_arr, g_classify_test_message, b_debug=True )

# Section B - Set up your mural

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-17_llm-cluster_03.png" width="50%" title="Image of a mural" />

<a id="step4"></a>
## Step 4: Set up MURAL prerequisites
- 4.1 Create empty, sample mural
- 4.2 Collect mural ID
- 4.3 Get Oauth token
- 4.4 Populate mural

### 4.1 Create empty, sample mural
In the MURAL web interface, create a new, empty mural.

### 4.2 Collect mural ID
You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [17]:
g_mural_id = ""

### 4.3 Collect OAuth token

In [18]:
g_auth_token = ""

### 4.4 Populate mural

In [19]:
import urllib

url = "https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/murals/sample-17_llm-cluster.json"
response = urllib.request.urlopen( url )
encoding = response.info().get_content_charset( "utf8" )
sample_widgets_arr = json.loads( response.read().decode( encoding ) )

In [20]:
import requests
import copy

def putWidget( auth_token, mural_id, widget ):
    # https://developers.mural.co/public/reference/createstickynote
    # https://developers.mural.co/public/reference/createtextbox
    widget_type = "textbox" if ( "text" == widget["type"] ) else widget["type"]
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/" + re.sub( "\s+", "-", widget_type )
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = copy.deepcopy( widget )
    if "id" in parms:
        del parms["id"]
    if "type" in parms:
        del parms["type"]
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ " + widget["id"] + " ] " + msg )

In [21]:
import time

time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the change
# ...

for widget in sample_widgets_arr:
    putWidget( g_auth_token, g_mural_id, widget )
print( "Done!" )

Done!


<a id="step5"></a>
## Step 5: Read feedback from your mural
- 5.1 Read widgets from the mural
- 5.2 Establish coordinates of class boxes

### 5.1 Read widgets from the mural

In [22]:
def listWidgets( auth_token, mural_id ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [23]:
g_widgets_arr = listWidgets( g_auth_token, g_mural_id )

In [ ]:
print( json.dumps( g_widgets_arr, indent=3 ) )

### 5.2 Establish coordinates of boxes
In the mural, there are three boxes, one box for each theme the LLM generates.

To be able to move the sticky notes into the correct box by theme, we need to know the coordinates of the boxes.

In [25]:
def getWidgetsByType( widgets_arr ):
    boxes_arr = []
    stickies_arr = []
    for widget in widgets_arr:
        obj = { "x" : widget["x"],
                "y" : widget["y"],
                "height" : widget["height"],
                "width"  : widget["width"] }
        if( "shape" == widget["type"] ):
            boxes_arr.append( obj )
        elif( "sticky note" == widget["type"] ):
            obj["id"]     = widget["id"]
            obj["text"]   = widget["text"]
            obj["height"] = widget["height"]
            obj["width"]  = widget["width"]
            obj["x_org"]  = widget["x"]
            obj["y_org"]  = widget["y"]
            stickies_arr.append( obj )
    return boxes_arr, stickies_arr

def getMuralObjects( widgets_arr ):
    boxes_arr, stickies_arr = getWidgetsByType( widgets_arr )
    print( "Boxes:" )
    for i in range( len( boxes_arr ) ):
        print( "Box " + str( i+1 ) + ": ( x=" + str( boxes_arr[i]["x"] ) + ", y=" + str( boxes_arr[i]["y"] ) + " )" )
    print( "\nStickies:")
    for sticky in stickies_arr:
        print( sticky["text"] )
    return boxes_arr, stickies_arr

In [26]:
g_boxes_arr, g_stickies_arr = getMuralObjects( g_widgets_arr )

Boxes:
Box 1: ( x=75, y=-450 )
Box 2: ( x=600, y=-450 )
Box 3: ( x=1100, y=-450 )

Stickies:
Our daily stand-ups take too long. Let's stop doing those.
Can we please stop having to sing the national anthem at the start of every day?!
Stop having all-hands meetings, because they don't add any value.
Stop forcing everyone to attend the all-hands in person
I like the "updates" Slack channel - very efficient
Loving the on-site day care
Please let's continue the THINK-Thursday meeting-free afternoon!
The monthly performance awards are a nice recognition
I think we should record our meetings so they could be played back
At my friend's company, they have a lottery for plum parking spots. We should do something similar!
We should take meeting minutes
Because of these power outages, we should invest in uninterrupted power supply infrastructure


# Section C - Cluster sticky notes in the mural

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-17_llm-cluster_01.png" width="50%" title="Image of a mural" />

<a id="step6"></a>
## Step 6: Identify top themes in sticky notes

In [27]:
def identifyThemesInStickies( model_id, prompt_parameters, stickies_arr, b_debug=False ):
    messages_arr = []
    for sticky in stickies_arr:
        messages_arr.append( sticky["text"] )
    class_names_arr = identifyThemes( model_id, prompt_parameters, messages_arr, b_debug )
    return class_names_arr

In [28]:
g_class_names_arr = identifyThemesInStickies( g_themes_model_id, g_themes_prompt_parameters, g_stickies_arr, b_debug=True )

print( json.dumps( g_class_names_arr, indent=3 ) )

Prompt:
--------------------------START
The following meals fall into three broad themes:
- Spaghetti with meatballs
- Vegetarian sub
- Shrimp pad thai
- Fish fingers
- Falafel
- Steak and kidney pie

Three themes in the list of meals:
- Meat
- Vegetarian
- Seafood


The following animals fall into three broad themes:
- Cow
- Chicken
- Dog
- Giraffe
- Gerbil
- Elephant

Three themes in the list of animals:
- Pet
- Farm
- Wild


The following process feedback ideas fall into three broad themes:
- Our daily stand-ups take too long. Let's stop doing those.
- Can we please stop having to sing the national anthem at the start of every day?!
- Stop having all-hands meetings, because they don't add any value.
- Stop forcing everyone to attend the all-hands in person
- I like the "updates" Slack channel - very efficient
- Loving the on-site day care
- Please let's continue the THINK-Thursday meeting-free afternoon!
- The monthly performance awards are a nice recognition
- I think we should rec

<a id="step7"></a>
## Step 7: Classify sticky notes

In [29]:
def classifyStickies( class_names_arr, stickies_arr ):
    for sticky in stickies_arr:
        class_name = classifyMessage( g_classify_model_id, g_classify_prompt_parameters, class_names_arr, sticky["text"] )
        sticky["class_name"] = class_name
    sorted_stickies_arr = sorted( stickies_arr, key=lambda d: d["class_name"] )
    print( "Results:\n" )
    for sticky in sorted_stickies_arr:
        class_name = "{:<10}".format( sticky["class_name"] )
        print( class_name + ": " + sticky["text"] )

In [30]:
classifyStickies( g_class_names_arr, g_stickies_arr )

Results:

Communication: Stop forcing everyone to attend the all-hands in person
Communication: I like the "updates" Slack channel - very efficient
Communication: We should take meeting minutes
Culture   : Can we please stop having to sing the national anthem at the start of every day?!
Culture   : Loving the on-site day care
Culture   : Please let's continue the THINK-Thursday meeting-free afternoon!
Culture   : The monthly performance awards are a nice recognition
Culture   : At my friend's company, they have a lottery for plum parking spots. We should do something similar!
Efficiency: Our daily stand-ups take too long. Let's stop doing those.
Efficiency: Stop having all-hands meetings, because they don't add any value.
Efficiency: I think we should record our meetings so they could be played back
Efficiency: Because of these power outages, we should invest in uninterrupted power supply infrastructure


<a id="step8"></a>
## Step 8: Move sticky notes into labelled boxes in the mural
- 8.1 Label boxes with class names
- 8.2 Move stickies into boxes by class name

### 8.1 Label boxes with class names

In [31]:
def labelPosition( box ):
    x = box["x"] + 20
    y = box["y"] + 20
    return x, y

def labelWidget( x, y, class_name ):
    widget = { "height": 75,
               "id": "4",
               "width": 400,
               "x": x,
               "y": y,
               "style": {
                  "backgroundColor": "#FFFFFF00",
                  "font": "proxima-nova",
                  "fontSize": 40,
                  "textAlign": "left"
               },
               "text": class_name,
               "type": "text" }
    return widget
    
def addBoxLabel( auth_token, mural_id, box, class_name ):
    x, y = labelPosition( box )
    widget = labelWidget( x, y, class_name )
    putWidget( auth_token, mural_id, widget )
    box["top_free_space"] = widget["y"] + widget["height"] + 20
    
def labelBoxes( auth_token, mural_id, boxes_arr, class_names_arr ):
    for i in range( len( boxes_arr ) ):
        box = boxes_arr[i]
        class_name = class_names_arr[i]
        box["class_name"] = class_name
        addBoxLabel( auth_token, mural_id, box, class_name )

In [32]:
import time

time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the change
# ...

labelBoxes( g_auth_token, g_mural_id, g_boxes_arr, g_class_names_arr )

### 8.2 Move stickies into boxes by class name

In [33]:
import random

def moveSticky( auth_token, mural_id, sticky_id, new_x, new_y ):
    # https://developers.mural.co/public/reference/updatestickynote
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/sticky-note/" + sticky_id
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = { "x" : new_x, "y" : new_y }
    response = requests.request( "PATCH", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ " + sticky_id + " ] " + msg )

def moveStickyToBox( auth_token, mural_id, sticky, box ):
    y = box["top_free_space"]
    min_x = box["x"]
    max_x = box["x"] + box["width"] - ( 0.8 * sticky["width"] )
    x = random.uniform( min_x, max_x )
    moveSticky( auth_token, mural_id, sticky["id"], x, y )
    return x, y

def boxesJSON( boxes_arr ):
    boxes_json = {}
    for box in boxes_arr:
        class_name = box["class_name"]
        boxes_json[ class_name ] = box
    return boxes_json

def moveStickiesToClassBoxes( auth_token, mural_id, boxes_arr, stickies_arr ):
    boxes_json = boxesJSON( boxes_arr )
    for sticky in stickies_arr:
        class_name = sticky["class_name"]
        box = boxes_json[ class_name ]
        new_x, new_y = moveStickyToBox( auth_token, mural_id, sticky, box )
        box["top_free_space"] = new_y + sticky["height"] + 10
    print( "Done!" )

In [34]:
time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the change
# ...

moveStickiesToClassBoxes( g_auth_token, g_mural_id, g_boxes_arr, g_stickies_arr )

Done!


Something like this:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-17_llm-cluster_04.gif" width="50%" title="Image of a mural" />